In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
from datetime import datetime, timedelta
import sys

# Add src directory to path
sys.path.append('../src')

# Import custom modules
from technical_indicators import TechnicalIndicators, add_technical_indicators

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully!")

## 1. Download Stock Price Data

We'll use yfinance to download historical stock data for analysis.

In [ ]:
# Define stocks to analyze - select major stocks from our news analysis
# These are typically the most covered stocks in financial news

STOCKS = ['AAPL', 'TSLA', 'AMZN', 'GOOGL', 'MSFT', 'META', 'NVDA', 'AMD']
START_DATE = '2020-01-01'
END_DATE = '2024-11-27'

print(f"📈 Downloading stock data for: {', '.join(STOCKS)}")
print(f"📅 Date range: {START_DATE} to {END_DATE}")

In [ ]:
# Download data for each stock
stock_data = {}

for ticker in STOCKS:
    print(f"\nDownloading {ticker}...", end=" ")
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=START_DATE, end=END_DATE)
        
        if not df.empty:
            df['Stock'] = ticker
            stock_data[ticker] = df
            print(f"✅ {len(df)} days of data")
        else:
            print(f"❌ No data available")
    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✅ Successfully downloaded data for {len(stock_data)} stocks")

In [ ]:
# Display sample data
if stock_data:
    sample_ticker = STOCKS[0]
    print(f"\n=== Sample Data for {sample_ticker} ===")
    print(stock_data[sample_ticker].head(10))
    
    print(f"\n=== Data Info ===")
    print(stock_data[sample_ticker].info())

## 2. Data Preparation and Basic Statistics

In [ ]:
# Calculate basic statistics for each stock
print("=== STOCK DATA SUMMARY ===\n")

for ticker, df in stock_data.items():
    print(f"\n{ticker}:")
    print(f"  Date range: {df.index.min().date()} to {df.index.max().date()}")
    print(f"  Trading days: {len(df)}")
    print(f"  Starting price: ${df['Close'].iloc[0]:.2f}")
    print(f"  Ending price: ${df['Close'].iloc[-1]:.2f}")
    
    total_return = ((df['Close'].iloc[-1] - df['Close'].iloc[0]) / df['Close'].iloc[0]) * 100
    print(f"  Total return: {total_return:.2f}%")
    
    avg_volume = df['Volume'].mean()
    print(f"  Average volume: {avg_volume:,.0f}")

In [ ]:
# Visualize closing prices for all stocks
plt.figure(figsize=(16, 8))

for ticker, df in stock_data.items():
    # Normalize to starting price for comparison
    normalized_price = (df['Close'] / df['Close'].iloc[0]) * 100
    plt.plot(df.index, normalized_price, label=ticker, linewidth=2)

plt.title('Normalized Stock Price Comparison (Base = 100)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Normalized Price', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 This chart shows relative performance with all stocks starting at 100")

## 3. Calculate Technical Indicators

We'll calculate various technical indicators for each stock:
- **Moving Averages**: SMA (20, 50), EMA (12, 26)
- **RSI**: Relative Strength Index (14-day)
- **MACD**: Moving Average Convergence Divergence
- **Bollinger Bands**: Upper, Middle, Lower bands
- **ATR**: Average True Range

In [ ]:
# Calculate technical indicators for each stock
stock_data_with_indicators = {}

print("Calculating technical indicators...\n")

for ticker, df in stock_data.items():
    print(f"Processing {ticker}...", end=" ")
    
    # Calculate indicators
    calculator = TechnicalIndicators(df)
    df_with_indicators = calculator.calculate_all_indicators()
    
    stock_data_with_indicators[ticker] = df_with_indicators
    print(f"✅ Done")

print(f"\n✅ Technical indicators calculated for all stocks")

In [ ]:
# Display sample with indicators
sample_ticker = STOCKS[0]
df_sample = stock_data_with_indicators[sample_ticker]

print(f"\n=== {sample_ticker} with Technical Indicators ===")
print(f"Total columns: {len(df_sample.columns)}")
print(f"\nColumn names:")
print(df_sample.columns.tolist())

print(f"\n=== Recent Data (Last 10 days) ===")
display_cols = ['Close', 'SMA_20', 'SMA_50', 'RSI_14', 'MACD', 'Daily_Return']
print(df_sample[display_cols].tail(10))

## 4. Visualize Technical Indicators

### 4.1 Moving Averages

In [ ]:
def plot_moving_averages(ticker, df, figsize=(16, 8)):
    """Plot price with moving averages"""
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot price and moving averages
    ax.plot(df.index, df['Close'], label='Close Price', linewidth=2, color='black')
    ax.plot(df.index, df['SMA_20'], label='SMA 20', linewidth=1.5, alpha=0.7, linestyle='--')
    ax.plot(df.index, df['SMA_50'], label='SMA 50', linewidth=1.5, alpha=0.7, linestyle='--')
    ax.plot(df.index, df['EMA_12'], label='EMA 12', linewidth=1, alpha=0.6)
    ax.plot(df.index, df['EMA_26'], label='EMA 26', linewidth=1, alpha=0.6)
    
    ax.set_title(f'{ticker} - Price with Moving Averages', fontsize=16, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Price ($)', fontsize=12)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot for first stock
ticker = STOCKS[0]
fig = plot_moving_averages(ticker, stock_data_with_indicators[ticker])
plt.show()

### 4.2 RSI (Relative Strength Index)

In [ ]:
def plot_rsi(ticker, df, figsize=(16, 10)):
    """Plot price and RSI"""
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True, 
                                    gridspec_kw={'height_ratios': [3, 1]})
    
    # Price chart
    ax1.plot(df.index, df['Close'], linewidth=2, color='black', label='Close Price')
    ax1.set_title(f'{ticker} - Price and RSI Analysis', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # RSI chart
    ax2.plot(df.index, df['RSI_14'], linewidth=2, color='purple', label='RSI (14)')
    ax2.axhline(y=70, color='r', linestyle='--', label='Overbought (70)', alpha=0.7)
    ax2.axhline(y=30, color='g', linestyle='--', label='Oversold (30)', alpha=0.7)
    ax2.fill_between(df.index, 30, 70, alpha=0.1, color='gray')
    ax2.set_ylabel('RSI', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_ylim(0, 100)
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot for first stock
ticker = STOCKS[0]
fig = plot_rsi(ticker, stock_data_with_indicators[ticker])
plt.show()

# Analyze RSI signals
df = stock_data_with_indicators[ticker]
overbought = (df['RSI_14'] > 70).sum()
oversold = (df['RSI_14'] < 30).sum()
print(f"\n📊 {ticker} RSI Analysis:")
print(f"  Overbought days (RSI > 70): {overbought} ({overbought/len(df)*100:.1f}%)")
print(f"  Oversold days (RSI < 30): {oversold} ({oversold/len(df)*100:.1f}%)")

### 4.3 MACD (Moving Average Convergence Divergence)

In [ ]:
def plot_macd(ticker, df, figsize=(16, 10)):
    """Plot price and MACD"""
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True,
                                    gridspec_kw={'height_ratios': [3, 1]})
    
    # Price chart
    ax1.plot(df.index, df['Close'], linewidth=2, color='black', label='Close Price')
    ax1.set_title(f'{ticker} - Price and MACD Analysis', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # MACD chart
    ax2.plot(df.index, df['MACD'], linewidth=2, label='MACD', color='blue')
    ax2.plot(df.index, df['MACD_Signal'], linewidth=2, label='Signal', color='red')
    ax2.bar(df.index, df['MACD_Hist'], label='Histogram', alpha=0.3, color='gray')
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax2.set_ylabel('MACD', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot for first stock
ticker = STOCKS[0]
fig = plot_macd(ticker, stock_data_with_indicators[ticker])
plt.show()

# Analyze MACD signals
df = stock_data_with_indicators[ticker]
bullish = ((df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1))).sum()
bearish = ((df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1))).sum()
print(f"\n📊 {ticker} MACD Analysis:")
print(f"  Bullish crossovers: {bullish}")
print(f"  Bearish crossovers: {bearish}")

### 4.4 Bollinger Bands

In [ ]:
def plot_bollinger_bands(ticker, df, figsize=(16, 8)):
    """Plot price with Bollinger Bands"""
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot price and Bollinger Bands
    ax.plot(df.index, df['Close'], label='Close Price', linewidth=2, color='black')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', linewidth=1, 
           alpha=0.7, linestyle='--', color='red')
    ax.plot(df.index, df['BB_Middle'], label='Middle Band (SMA 20)', linewidth=1, 
           alpha=0.7, linestyle='--', color='blue')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', linewidth=1, 
           alpha=0.7, linestyle='--', color='green')
    
    # Fill between bands
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1, color='gray')
    
    ax.set_title(f'{ticker} - Bollinger Bands', fontsize=16, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Price ($)', fontsize=12)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot for first stock
ticker = STOCKS[0]
fig = plot_bollinger_bands(ticker, stock_data_with_indicators[ticker])
plt.show()

# Analyze Bollinger Band breaches
df = stock_data_with_indicators[ticker]
upper_breach = (df['Close'] > df['BB_Upper']).sum()
lower_breach = (df['Close'] < df['BB_Lower']).sum()
print(f"\n📊 {ticker} Bollinger Bands Analysis:")
print(f"  Upper band breaches: {upper_breach} ({upper_breach/len(df)*100:.1f}%)")
print(f"  Lower band breaches: {lower_breach} ({lower_breach/len(df)*100:.1f}%)")

### 4.5 Comprehensive Dashboard

In [ ]:
def plot_comprehensive_dashboard(ticker, df, recent_days=252):
    """Create comprehensive technical analysis dashboard"""
    
    # Use recent data for clarity
    df_recent = df.tail(recent_days)
    
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(4, 2, hspace=0.3, wspace=0.3)
    
    # 1. Price with Moving Averages
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(df_recent.index, df_recent['Close'], label='Close', linewidth=2, color='black')
    ax1.plot(df_recent.index, df_recent['SMA_20'], label='SMA 20', linewidth=1.5, alpha=0.7)
    ax1.plot(df_recent.index, df_recent['SMA_50'], label='SMA 50', linewidth=1.5, alpha=0.7)
    ax1.set_title(f'{ticker} - Technical Analysis Dashboard', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price ($)')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # 2. Volume
    ax2 = fig.add_subplot(gs[1, :], sharex=ax1)
    ax2.bar(df_recent.index, df_recent['Volume'], alpha=0.7, color='steelblue')
    ax2.plot(df_recent.index, df_recent['Volume_SMA_20'], color='red', 
            linewidth=2, label='Volume SMA 20')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    # 3. RSI
    ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
    ax3.plot(df_recent.index, df_recent['RSI_14'], linewidth=2, color='purple')
    ax3.axhline(y=70, color='r', linestyle='--', alpha=0.5)
    ax3.axhline(y=30, color='g', linestyle='--', alpha=0.5)
    ax3.fill_between(df_recent.index, 30, 70, alpha=0.1)
    ax3.set_ylabel('RSI')
    ax3.set_ylim(0, 100)
    ax3.set_title('RSI (14)')
    ax3.grid(True, alpha=0.3)
    
    # 4. MACD
    ax4 = fig.add_subplot(gs[2, 1], sharex=ax1)
    ax4.plot(df_recent.index, df_recent['MACD'], linewidth=2, label='MACD', color='blue')
    ax4.plot(df_recent.index, df_recent['MACD_Signal'], linewidth=2, label='Signal', color='red')
    ax4.bar(df_recent.index, df_recent['MACD_Hist'], alpha=0.3, color='gray')
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax4.set_ylabel('MACD')
    ax4.set_title('MACD')
    ax4.legend(loc='best', fontsize=9)
    ax4.grid(True, alpha=0.3)
    
    # 5. Daily Returns Distribution
    ax5 = fig.add_subplot(gs[3, 0])
    ax5.hist(df['Daily_Return'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='teal')
    ax5.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax5.set_xlabel('Daily Return (%)')
    ax5.set_ylabel('Frequency')
    ax5.set_title('Daily Returns Distribution')
    ax5.grid(True, alpha=0.3)
    
    # 6. Returns Statistics
    ax6 = fig.add_subplot(gs[3, 1])
    ax6.axis('off')
    
    # Calculate statistics
    returns = df['Daily_Return'].dropna()
    stats_text = f"""
    PERFORMANCE METRICS
    
    Total Return: {((df['Close'].iloc[-1]/df['Close'].iloc[0]-1)*100):.2f}%
    
    Daily Returns:
      Mean: {returns.mean():.3f}%
      Std Dev: {returns.std():.3f}%
      Sharpe (approx): {(returns.mean()/returns.std())*np.sqrt(252):.2f}
    
    Current Indicators:
      RSI: {df['RSI_14'].iloc[-1]:.1f}
      MACD: {df['MACD'].iloc[-1]:.2f}
      Price vs SMA20: {((df['Close'].iloc[-1]/df['SMA_20'].iloc[-1]-1)*100):.2f}%
      Price vs SMA50: {((df['Close'].iloc[-1]/df['SMA_50'].iloc[-1]-1)*100):.2f}%
    """
    
    ax6.text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
            verticalalignment='center')
    
    plt.tight_layout()
    return fig

# Create dashboards for selected stocks
for ticker in STOCKS[:3]:  # First 3 stocks
    print(f"\n{'='*60}")
    print(f"Creating dashboard for {ticker}")
    print('='*60)
    fig = plot_comprehensive_dashboard(ticker, stock_data_with_indicators[ticker])
    plt.show()

## 5. Multi-Stock Comparison

In [ ]:
# Compare RSI across stocks
fig, ax = plt.subplots(figsize=(16, 8))

for ticker in STOCKS:
    df = stock_data_with_indicators[ticker]
    df_recent = df.tail(252)  # Last year
    ax.plot(df_recent.index, df_recent['RSI_14'], label=ticker, linewidth=2, alpha=0.7)

ax.axhline(y=70, color='r', linestyle='--', label='Overbought', alpha=0.5)
ax.axhline(y=30, color='g', linestyle='--', label='Oversold', alpha=0.5)
ax.fill_between(df_recent.index, 30, 70, alpha=0.1, color='gray')

ax.set_title('RSI Comparison - All Stocks (Last Year)', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('RSI', fontsize=12)
ax.set_ylim(0, 100)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate correlation between stocks
print("=== CORRELATION ANALYSIS ===\n")

# Create returns matrix
returns_dict = {}
for ticker in STOCKS:
    returns_dict[ticker] = stock_data_with_indicators[ticker]['Daily_Return']

returns_df = pd.DataFrame(returns_dict).dropna()

# Calculate correlation
correlation_matrix = returns_df.corr()

print("Daily Returns Correlation Matrix:")
print(correlation_matrix.round(3))

# Visualize correlation
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
           center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Stock Returns Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 High correlation indicates stocks tend to move together")

## 6. Export Data with Technical Indicators

In [ ]:
# Save processed stock data
import os

output_dir = '../Data/stockData'
os.makedirs(output_dir, exist_ok=True)

for ticker, df in stock_data_with_indicators.items():
    output_path = f'{output_dir}/{ticker}_with_indicators.csv'
    df.to_csv(output_path)
    print(f"✅ Saved {ticker} data to {output_path}")

# Also save combined data
combined_df = pd.concat([df.assign(Stock=ticker) for ticker, df in stock_data_with_indicators.items()])
combined_path = f'{output_dir}/all_stocks_with_indicators.csv'
combined_df.to_csv(combined_path)
print(f"\n✅ Saved combined data to {combined_path}")

print(f"\n📦 Total records saved: {len(combined_df):,}")

## 7. Summary and Key Findings

In [ ]:
print("=" * 80)
print("TECHNICAL ANALYSIS - KEY FINDINGS")
print("=" * 80)

for ticker in STOCKS:
    df = stock_data_with_indicators[ticker]
    
    # Calculate metrics
    total_return = ((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100
    volatility = df['Daily_Return'].std()
    sharpe = (df['Daily_Return'].mean() / df['Daily_Return'].std()) * np.sqrt(252)
    current_rsi = df['RSI_14'].iloc[-1]
    
    # Determine trend
    if df['Close'].iloc[-1] > df['SMA_20'].iloc[-1] > df['SMA_50'].iloc[-1]:
        trend = "Strong Uptrend"
    elif df['Close'].iloc[-1] > df['SMA_20'].iloc[-1]:
        trend = "Uptrend"
    elif df['Close'].iloc[-1] < df['SMA_20'].iloc[-1] < df['SMA_50'].iloc[-1]:
        trend = "Strong Downtrend"
    else:
        trend = "Downtrend"
    
    # RSI status
    if current_rsi > 70:
        rsi_status = "Overbought"
    elif current_rsi < 30:
        rsi_status = "Oversold"
    else:
        rsi_status = "Neutral"
    
    print(f"\n{ticker}:")
    print(f"  📈 Total Return: {total_return:.2f}%")
    print(f"  📊 Volatility (Daily): {volatility:.3f}%")
    print(f"  💹 Sharpe Ratio: {sharpe:.2f}")
    print(f"  📉 Trend: {trend}")
    print(f"  🎯 RSI: {current_rsi:.1f} ({rsi_status})")
    print(f"  💰 Current Price: ${df['Close'].iloc[-1]:.2f}")

print("\n" + "=" * 80)

print("\n💡 KEY INSIGHTS:")
print("  • Technical indicators provide insights into momentum and trends")
print("  • Moving averages help identify support/resistance levels")
print("  • RSI and MACD signal potential overbought/oversold conditions")
print("  • Bollinger Bands measure volatility and potential breakouts")
print("  • These indicators are now ready for correlation with news sentiment")

print("\n" + "=" * 80)

## Conclusion

Successfully completed Task 2 - Technical Analysis:

### What We Accomplished:
1. ✅ Downloaded historical stock price data using yfinance
2. ✅ Calculated comprehensive technical indicators:
   - Moving Averages (SMA, EMA)
   - RSI (Relative Strength Index)
   - MACD (Moving Average Convergence Divergence)
   - Bollinger Bands
   - Daily Returns
   - Volume indicators
3. ✅ Created professional visualizations for each indicator
4. ✅ Built comprehensive technical analysis dashboards
5. ✅ Analyzed correlations between stocks
6. ✅ Exported data with all indicators for next task

### Key Learnings:
- Technical indicators reveal momentum, trends, and potential reversal points
- Different stocks show varying volatility and correlation patterns
- Indicators work best when used in combination
- Visual analysis helps identify patterns not obvious in raw data

### Next Steps (Task 3):
- Perform sentiment analysis on news headlines
- Align sentiment scores with stock price data
- Calculate correlations between news sentiment and stock returns
- Identify if positive/negative news correlates with price movements
- Analyze lagged effects of news on prices